# Pholly

Welcome to the Pholly interactive notebook - this is an introduction to using resilience patterns with F# and Pholly which is itself based on Polly.

## What is Pholly

Pholly provides a more natural way to use Polly from within F# and uses a number of F# language features to provide an expressive DSL type approach to using common resilience patterns. Currently the three Polly reactive resilience patterns are supported:

|Pattern|Purpose|
|-------|-------|
|Retry|Many faults are transient and may self-correct after a short delay and the retry pattern will attempt to retry the operation until it succeeds or a failure condition is met|
|Circuit Breaker|When a system is seriously struggling, failing fast is better than making users/callers wait. The circuit breaker can protect a faulting system from overloading and help it recover.|
|Fallback|Things can and do fail and it can be useful to provide a fallback|

## Pholly Principles

Pholly uses the F# Result<> type for its error control flow rather than the exceptions you normally see in Polly examples (which are often based on C#).

## Using Pholly

### Installing Pholly

You can add Pholly either as a single source file [Pholly.fs](https://github.com/JamesRandall/Pholly/blob/main/src/Pholly.fs) or as a [NuGet package Pholly](https://www.nuget.org/packages/Pholly/0.0.2) using the package manager of your choice.

### Retry Policy

Before getting started with the policies we'll just get started by setting up some common concerns: import the Pholly package and create some methods that will let us roll a six sided dice and test if its 5 or higher.

In [1]:
#!fsharp
#r "nuget: Pholly"

open Pholly
open Retry
open System

let log msg = printf "%s" msg

let random = Random((DateTime.UtcNow.Ticks % (Int32.MaxValue |> int64)) |> int32)
let rollDice () = random.Next(1,7)
let isSuccessfulDiceRoll diceRoll = if diceRoll >= 5 then diceRoll |> Ok else "Out of range" |> Error

Installed package Pholly version 0.0.5

Now we're going to wrap our dice roll in a retry policy and keep on retrying it until we roll a 5 or 6:

In [1]:
#!fsharp
let retryForever = Policy.retryForever [ ]
let result = (rollDice >> isSuccessfulDiceRoll) |> retryForever
sprintf "Success - returned %d" result |> log

Success - returned 6

Its not usually helpful to retry an operation forever and so the code below builds on our forever example shows a simple retry policy that will roll a dice and continue to retry until a 5 or higher is rolled or 10 errors occur. Before each retry it will print a message with the retry count. Note that now we are not retrying forever we have to still expect a possible Result.Error:

In [1]:
#!fsharp
let retryPolicy = Policy.retry [
    retry (upto 10<times>)
    beforeEachRetry (fun _ retryAttempt _ -> sprintf "Retrying attempt %d" retryAttempt |> log)
]

match (rollDice >> isSuccessfulDiceRoll) |> retryPolicy with
| Ok value -> sprintf "Success - returned %d" value |> log
| Error error -> sprintf "ERROR: %s" error |> log


Retrying attempt 1

Success - returned 6

The next example shows an asynchronous retry policy for an asynchronous workload that also includes a back off sequence - each retry attempt will result in the next delay in the series. If their are more retries than their are specified intervals then the out of range retries will use the final interval.

In [1]:
#!fsharp
let retryPolicy = Policy.retryAsync [
    retry (upto 10<times>)
    withIntervalsOf [50<ms> ; 500<ms> ; 1000<ms>]
    beforeEachRetry (fun _ retryAttempt _ -> sprintf "Retrying attempt %d" retryAttempt |> log)
]

match (async { return (rollDice () |> isSuccessfulDiceRoll) }) |> retryPolicy |> Async.RunSynchronously with
| Ok value -> sprintf "Success - returned %d" value |> log
| Error error -> sprintf "ERROR: %s" error |> log


Retrying attempt 1

Retrying attempt 2

Retrying attempt 3

Retrying attempt 4

Success - returned 6

### Circuit Breaker

The circuit breaker policy is a good way to keep a failing system space to recover - for example their's no point pounding on an API endpoint that is already overloaded and returning errors as that will likely hinder its recovery.

First we'll begin by opening the circuit breaker module and creating a little helper to output the result of our circuit breaker protected operations:

In [1]:
#!fsharp
open CircuitBreaker

let outputBreakerResult result = match result with | Ok s -> log s | Error e -> log (sprintf "Error: %s\n" e)

Next we'll create a circuit breaker such that if it returns 3 consecutive errors it will trip into the open state. When a circuit breaker is open, like an electric circuit, nothing will pass through it. We then use the breaker to make a series of calls. We'll start with a successful call (return an Ok) and then we'll generate 4 errors. Our circuit breaker will run the first 3 erroring operations and return the error result to us. The 3rd call will trip the breaker and so our fourth operation will never be called:

In [1]:
#!fsharp
// NOTE: would love feedback on if people prefer these as a three part tuple or would prefer a interface type approach e.g.
// let breaker = Policy.circuitBreaker [...]
// breaker.execute (...)
let execute,reset,isolate =
    Policy.circuitBreaker [
        breakOn 3<consecutiveErrors>
        whenCircuitIsOpenReturn ("Circuit is open" |> Error)
    ]

execute (fun () -> "Success" |> Ok) |> outputBreakerResult
execute (fun () -> "Consecutive error 1" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error 2" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error 3 - should trip breaker" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error 4 - should not be called, instead we should get 'Circuit is open'" |> Error) |> outputBreakerResult

Success

Error: Consecutive error 1


Error: Consecutive error 2


Error: Consecutive error 3 - should trip breaker


Error: Circuit is open


The next example breaks up our 3 consecutive errors with another successful result. This means the circuit breaker never trips as their are never more than 2 consecutive calls:

In [1]:
#!fsharp
execute (fun () -> "Success" |> Ok) |> outputBreakerResult
execute (fun () -> "Consecutive error a.1" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error a.2" |> Error) |> outputBreakerResult
execute (fun () -> "Success" |> Ok) |> outputBreakerResult
execute (fun () -> "Consecutive error b.1" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error b.2" |> Error) |> outputBreakerResult

Error: Circuit is open


Error: Circuit is open


Error: Circuit is open


Error: Circuit is open


Error: Circuit is open


Error: Circuit is open


In the next example we'll reset the open breaker so we can continue to make successful function calls:

In [1]:
#!fsharp
reset() // we will still be open from the previous demo
execute (fun () -> "Success" |> Ok) |> outputBreakerResult
execute (fun () -> "Consecutive error 1" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error 2" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error 3 - should trip breaker" |> Error) |> outputBreakerResult
execute (fun () -> "Consecutive error 4 - should not be called, instead we should get 'Circuit is open'" |> Error) |> outputBreakerResult
reset ()
execute (fun () -> "Success" |> Ok) |> outputBreakerResult

Success

Error: Consecutive error 1


Error: Consecutive error 2


Error: Consecutive error 3 - should trip breaker


Error: Circuit is open


Success

Next we'll show how you can manually force a circuit breaker open by using the isolate method. Immediately after calling isolate we will find our calls blocked by the breaker:

In [1]:
#!fsharp
execute (fun () -> "Success" |> Ok) |> outputBreakerResult
isolate ()
execute (fun () -> "Will not be called" |> Ok) |> outputBreakerResult

Success

Error: Circuit is open


Circuit breaks will automatically reset after a given duration - by default that is 1 minute with Pholly but you can change it by constructing a circuit breaker with the resetAfter option - this takes &lt;ms&gt; but their are simple helper functions to convert to other units. In this case we'll set the breaker to after a reset duration of 30 seconds:

In [1]:
#!fsharp
let execute,_,_ =
    Policy.circuitBreaker [
        breakOn 3<consecutiveErrors>
        resetAfter (30 |> seconds)
        whenCircuitIsOpenReturn ("Circuit is open" |> Error)
    ]

execute (fun () -> "Success" |> Ok) |> outputBreakerResult

Success

We can also use circuit breakers with async operations:

In [1]:
#!fsharp
let outputBreakerResultAsync result = async { match! result with | Ok s -> log s | Error e -> log (sprintf "Error: %s\n" e) }

let executeAsync,_,_ =
    Policy.circuitBreakerAsync [
        breakOn 3<consecutiveErrors>
        whenCircuitIsOpenReturn ("Circuit is open" |> Error)
    ]

async {
    do! async { return "Success" |> Ok } |> executeAsync |> outputBreakerResultAsync
    do! async { return "Error" |> Error } |> executeAsync |> outputBreakerResultAsync
    do! async { return "Error" |> Error } |> executeAsync |> outputBreakerResultAsync
    do! async { return "Error" |> Error } |> executeAsync |> outputBreakerResultAsync
    do! async { return "Success" |> Ok } |> executeAsync |> outputBreakerResultAsync    
} |> Async.RunSynchronously

Success

Error: Error


Error: Error


Error: Error


Error: Circuit is open


### Fallback

When all else fails it can be useful to provide a fallback result and you can do this with the fallback policy. The example below will fallback to the meaning of life if an error occurs:

In [1]:
#!fsharp
let fallbackPolicy = Policy.fallbackWith 42
(fun () -> "Gone wrong" |> Error) |> fallbackPolicy |> sprintf "Fallback result: %d" |> log
(fun () -> 1701 |> Ok) |> fallbackPolicy |> sprintf "Fallback result: %d" |> log

Fallback result: 42

Fallback result: 1701

In [1]:
#!fsharp


### Policy Composition

Its possible to combine policies to provide protection in depth by using the ||> and |||> operators (for none-async and async respectively). For example in the below we run code inside a circuit breaker that is wrapped in a retry block and ultimate provides a fallback value when all else fails:

In [1]:
#!fsharp
let failingWorkload = fun () -> "Always going wrong to show error control flow" |> Error
let breakerPolicy,_,_ = Policy.circuitBreaker [
    breakOn 3<consecutiveErrors>
    whenCircuitIsOpenReturn ("Circuit is open, execution blocked" |> Error)
    whenCircuitIsOpened (fun _ _ _ -> "Circuit opening\n" |> log)
  ]
let retryPolicy = Policy.retry [
    retry (upto 10<times>)
    beforeEachRetry (fun r _ _ -> sprintf "Retrying after error: %s \n" r |> log)
]
let fallbackPolicy = Policy.fallbackWith 42  
  
let result = failingWorkload |> (breakerPolicy ||> retryPolicy ||> fallbackPolicy)
  
sprintf "Final result %d" result |> log

Retrying after error: Always going wrong to show error control flow 


Retrying after error: Always going wrong to show error control flow 


Circuit opening


Retrying after error: Always going wrong to show error control flow 


Retrying after error: Circuit is open, execution blocked 


...

Retrying after error: Circuit is open, execution blocked 


Retrying after error: Circuit is open, execution blocked 


Retrying after error: Circuit is open, execution blocked 


Retrying after error: Circuit is open, execution blocked 


Final result 42

The async version is pretty much the same but note the use of the |||> operator for composition and in this case I've composed the policy into a reusable value:

In [1]:
#!fsharp
async {  
    let failingAsyncWorkload = async { return "Always going wrong to show error control flow" |> Error }
    let asyncBreakerPolicy,_,_ = Policy.circuitBreakerAsync [
        breakOn 3<consecutiveErrors>
        whenCircuitIsOpenReturn ("Circuit is open, execution blocked" |> Error)
        whenCircuitIsOpened (fun _ _ _ -> "Circuit opening\n" |> log)
    ]
    let asyncRetryPolicy = Policy.retryAsync [
        retry (upto 10<times>)
        beforeEachRetry (fun r _ _ -> sprintf "Retrying after error: %s \n" r |> log)
    ]
    let asyncFallbackPolicy = Policy.fallbackAsyncWith 42  
    let composedPolicy = asyncBreakerPolicy ||> asyncRetryPolicy ||> asyncFallbackPolicy

    let! result = failingAsyncWorkload |> composedPolicy
    
    sprintf "Final result %d" result |> log
    return ()
} |> Async.RunSynchronously

Error: input.fsx (13,44)-(13,55) typecheck error Type mismatch. Expecting a
    '(unit -> Async<Result<obj,string>>) -> obj'    
but given a
    'Async<Result<obj,string>> -> Async<Result<obj,string>>'    
The type 'unit -> Async<Result<obj,string>>' does not match the type 'Async<Result<obj,string>>'
input.fsx (13,60)-(13,74) typecheck error Type mismatch. Expecting a
    '(unit -> obj) -> Async<int>'    
but given a
    'Async<Result<int,obj>> -> Async<int>'    
The type 'unit -> obj' does not match the type 'Async<Result<int,obj>>'

## Thanks

Many thanks to the creators of [Polly](https://github.com/App-vNext/Polly) itself - I've used it in many production systems and I've used some of their descriptions of the patterns here.

And thanks to [Scott Wlaschin](https://twitter.com/ScottWlaschin) for the name. If I have a kryptonite its naming things.